# Machine Learning Project

## Setup

In [ ]:
from base import *

# Import for cross-validation.
from grid_search import cv_grid_search
from simulated_annealing import SimulatedAnnealing, cv_simulated_annealing

logging.basicConfig(
	filename="main.log",
	level=logging.INFO,
	format="%(asctime)s - %(message)s",
	datefmt="%Y-%m-%d %H:%M:%S"
)

def train_model(model_type, X_train, y_train, **kwargs):
	model_class, default_params = MODEL_TYPE_TO_CLASS_TO_HYPERPARAMETER_MAP[model_type]
	params = {**default_params, **kwargs}
	model = model_class(**params)
	model.fit(X_train, y_train, plot=True)
	return model

def predict_model(model, X):
	return model.predict(X)

def generate_predictions(model_type, **kwargs):
	start_time = time.time()
	model = train_model(model_type, np.array(X_train), np.array(y_train), **kwargs)
	predictions = predict_model(model, np.array(X_test))
	output_dir = f"./predictions/{model_type}/"
	os.makedirs(output_dir, exist_ok=True)
	file_name = os.path.join(output_dir, f"{kwargs}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

## Task 1

### logreg Model

In [ ]:
def σ(z): return 1 / (1 + np.exp(-z))
def bce_loss(y, ŷ): return (-1/(y.shape[0])) * np.sum(y * np.log(ŷ) + (1 - y) * np.log(1 - ŷ))

# Return dw and db, for some X, y, ŷ, w, R, and λ.
def gradients_logreg(X, y, ŷ, w, R=None, λ=0):
	m, _ = X.shape
	dw = 1/m * np.dot(X.T, (ŷ - y))
	db = 1/m * np.sum(ŷ - y)
	if R == "L2":
		dw += λ * w / m
	elif R == "L1":
		dw += λ * np.sign(w) / m
	return dw, db

# Return (w, b) from gradient descent on X_train and y_train, for some τ, α, G, β, R, and λ.
def train_logreg(X_train, y_train, τ=1000, α=0.1, G="mini-batch", β=128, R=None, λ=0):
	m, n = X_train.shape
	w, b = np.zeros((n, 1)), 0
	for epoch in range(τ):
		if G == "full-batch":
			X_batch, y_batch = X_train, y_train
			ŷ = σ(np.dot(X_batch, w) + b)
			dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
			w, b = w - α*dw, b - α*db
		elif G == "mini-batch":
			for i in range(0, m, β):
				X_batch, y_batch = X_train[i:i+β], y_train[i:i+β]
				ŷ = σ(np.dot(X_batch, w) + b)
				dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
				w, b = w - α*dw, b - α*db
		elif G == "stochastic":
			for i in range(m):
				X_batch, y_batch = X_train[i:i+1], y_train[i:i+1]
				ŷ = σ(np.dot(X_batch, w) + b)
				dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
				w, b = w - α*dw, b - α*db
	return w, b

# Return array of predictions, where each prediction is 1 if corresponding ŷ entry > 0.5, and 0 otherwise.
def predict_logreg(wb_tuple, X):
	w, b = wb_tuple
	ŷ = σ(np.dot(X, w) + b)
	return np.array([1 if p > 0.5 else 0 for p in ŷ])

# Train model, make predictions, and save predictions to CSV file.
def generate_predictions_logreg(τ=1000, α=0.1, G="mini-batch", β=128, R=None, λ=0):
	start_time = time.time()
	w, b = train_logreg(np.array(X_train), np.array(y_train), τ, α, G, β, R, λ)
	predictions = predict_logreg((w, b), np.array(X_test))
	os.makedirs("./predictions/logreg/", exist_ok=True)
	file_name = os.path.join("./predictions/logreg/", f"τ={τ},α={α},G={G},β={β},R={R},λ={λ}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

## Task 2

In [ ]:
def apply_pca(x):
	scaler = StandardScaler()
	X_train_scaled = scaler.fit_transform(X_train)
	X_test_scaled = scaler.transform(X_test)
	if 0 <= x <= 1:
		# x is variance threshold.
		pca = PCA(n_components=None)
		pca.fit(X_train_scaled)
		c = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= x) + 1
		v = x
	else:
		# x is number of components.
		pca = PCA(n_components=x)
		pca.fit(X_train_scaled)
		c = x
		v = sum(pca.explained_variance_ratio_)
	# Transform train and test datasets.
	X_train_pca = pca.transform(X_train_scaled)
	X_test_pca = pca.transform(X_test_scaled)
	return X_train_pca, X_test_pca, c, v

def train_SKLknn(X_train, y_train, k=5, W="uniform", p=2, m="minkowski"):
	model = KNeighborsClassifier(
		n_neighbors=k,
		weights=W,
		p=p,
		metric=m,
		n_jobs=-1
	)
	model.fit(X_train, y_train)
	return model

def predict_SKLknn(model, X): return model.predict(X)

# Train model, make model predictions, and save model predictions to CSV file.
def generate_predictions_pcaknn(x):
	start_time = time.time()
	X_train_pca, X_test_pca, c, v = apply_pca(x)
	model = train_SKLknn(X_train_pca, y_train, k=2)
	predictions = predict_SKLknn(model, X_test_pca)
	os.makedirs("./predictions/pcaknn/", exist_ok=True)
	file_name = os.path.join("./predictions/pcaknn/", f"pcaknn(ρ={c},ν={v:.2f}).csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### pcaknn Area

In [ ]:
generate_predictions_pcaknn(100)
generate_predictions_pcaknn(500)
generate_predictions_pcaknn(1000)
generate_predictions_pcaknn(2000)
generate_predictions_pcaknn(5000)

## Task 3

### SKLknn Area

In [ ]:
# generate_predictions(
# 	"SKLknn",
# 	n_neighbors=7,
# 	weights="distance",
# 	p=1,
# 	metric="minkowski"
# )

### SKLrf Area

In [ ]:
# generate_predictions(
# 	"SKLrf",
# 	n_estimators=150,
# 	criterion="entropy",
# 	max_depth=10,
# 	min_samples_split=5,
# 	min_samples_leaf=2,
# 	max_leaf_nodes=50,
# 	random_state=None
# )

### SKLsvm Area

In [ ]:
# generate_predictions(
# 	"SKLsvm",
# 	C=0.5,
# 	kernel="linear",
# 	degree=3,
# 	gamma="auto",
# 	max_iter=1000,
# 	random_state=None
# )

### SKLgb Area

In [ ]:
# generate_predictions(
# 	"SKLgb",
# 	learning_rate=0.05,
# 	n_estimators=200,
# 	subsample=0.8,
# 	min_samples_split=3,
# 	min_samples_leaf=2,
# 	max_depth=5,
# 	random_state=None,
# 	max_leaf_nodes=50
# )

### SKLhgb Area

In [ ]:
# generate_predictions(
# 	"SKLhgb",
# 	learning_rate=0.01,
# 	max_iter=200,
# 	max_leaf_nodes=50,
# 	max_depth=5,
# 	min_samples_leaf=10,
# 	random_state=None
# )

### XGBgb Area

In [ ]:
# generate_predictions(
# 	"XGBgb",
# 	n_estimators=150,
# 	max_depth=4,
# 	max_leaves=15,
# 	learning_rate=0.05,
# 	objective="binary:logistic",
# 	booster="gbtree",
# 	subsample=0.9,
# 	reg_alpha=0.1,
# 	reg_lambda=1.2,
# 	random_state=None
# )

### CBgb Area

In [ ]:
# generate_predictions(
# 	"CBgb",
# 	learning_rate=0.03,
# 	subsample=0.9,
# 	max_depth=6,
# 	n_estimators=1000,
# 	reg_lambda=1.0,
# 	random_state=None
# )

### Cross-Validation Area

In [ ]:
grid = {
	"learning_rate": [0.08, 0.09, 0.1],
	"max_depth": [8, 9],
	"n_estimators": [500]
}
cv_grid_search("CBgb", grid, k=5)

In [ ]:
# grid = {
# 	"learning_rate": (0.06, 0.12),
# 	"max_depth": [12],
# 	"reg_lambda": (0.1, 1),
# }

# cv_simulated_annealing(model_name="CBgb", grid=grid)